---
title: "VO2-Modellfunktion während der Belastung"
format:
  html:
    resources: 
      - shinylive-sw.js
      
filters:
  - webr
  - shinylive
---

```{css}
#| echo: false
p {
  text-align: justify
}
caption {
  text-align: justify;
}

```





# Kinetik des Sauerstoffvolumenstroms

## Monoexponentielle Modellfunktion des Sauerstoffvolumenstroms in Abhängigkeit von der Zeit während der Belastung

Der Anstieg des Sauerstoffvolumenstroms ($\dot{V}O_{2}$) zu Beginn einer leichten bis moderaten körperlichen Belastung auf dem Fahrradergometer unterhalb der Laktatschwelle (LT[^1]) wurde traditionell in der Literatur als monoexponentiell beschrieben, mit einer Amplitude von ungefähr 9–10 ml O~2~·W^-1^·min^-1^ [@Barstow1991; @Bearden2000]. Die Zeitkonstante $\tau_{\text{Start}}$ beschreibt die Geschwindigkeit, mit der der Sauerstoffvolumenstrom den Gleichgewichtszustand $\dot{V}O_{2,\text{gross, SS}}$ erreicht. Dies lässt sich mit der folgenden Formel darstellen [@Hill1924; @Henry1956; @Margaria1965; @Whipp1971; @Oezyner2001; @Francescato2003; @Clark2014]:

[^1]:Die LT beschreibt den Punkt, an dem das Blutlaktat bei zunehmender Belastungsintensität über das Ruheniveau hinaus zu akkumulieren beginnt [@Siestema2020]. 

$$
\dot{V}O_{2,\text{Belastung}}(t) = \dot{V}O_{2,\text{gross, SS}} \cdot \left(1 - e^{-\frac{t - TD}{\tau_{\text{Start}}}}\right) + \dot{V}O_{2,\text{Start}}
$$ {#eq-VO2_Start1}

Dabei beschreibt $\dot{V}O_{2,\text{Start}}$ den Sauerstoffvolumenstrom zu Beginn der Belastung, und $\dot{V}O_{2,\text{net, SS}}$ bezeichnet den Steady-State-Wert abzüglich des Ruhe-Sauerstoffvolumenstroms ($\dot{V}O_{2, \text{Ruhe}}$). In der folgenden Shiny-App wird ein Beispiel dieser Modellfunktion gezeigt, wie in @eq-VO2_Start1 beschrieben. Eigene $\dot{V}O_{2}$-Daten können hochgeladen und mittels nichtlinearer Regression an die Beispieldaten angepasst werden.

## Mono-exponentielle Shiny-App
```{shinylive-r}
#| standalone: true
#| viewerHeight: 900

library(shiny)
library(plotly)
library(minpack.lm)

# Beispieldaten
t_data <- c(0, 2.8, 5.3, 8.6, 10.4, 15.2, 19.7, 22.5, 24.5, 26.2, 28.2, 31.6, 33.5, 35.3, 37, 38.5, 40.3, 41.9, 43.5, 45.2, 46.8, 48.4, 50, 51.5, 53.3, 54.8, 56.5, 57.9, 59.5, 61.1, 62.9, 64.3, 65.8, 67.7, 69.1, 70.7, 72.4, 73.9, 75.7, 77.2, 78.9, 80.5, 82.7, 84.1, 85.5, 87, 88.6, 90.2, 91.5, 92.9, 94.3, 95.8, 97.6, 99.1, 100.5, 102.1, 103.5, 105.3, 106.6, 108.1, 109.6, 111.4, 112.8, 114.3, 115.8, 117.7, 119.1, 120.8, 122.3, 124.2, 125.7, 127.2, 128.8, 131.1, 132.5, 133.9, 135.4, 136.9, 138.4, 140, 141.3, 143, 144.5, 146.1, 147.6, 149, 150.5, 152.3, 153.9, 155.6, 156.9, 158.6, 160.1, 162.4, 163.7, 165.1)
VO2_data <- c(0.465, 0.34, 0.466, 0.503, 0.452, 0.687, 0.846, 0.963, 0.936, 1.125, 1.238, 1.541, 1.395, 1.685, 1.689, 1.622, 1.917, 1.732, 1.835, 1.801, 1.664, 1.744, 1.891, 1.85, 1.858, 1.89, 2.108, 1.962, 1.992, 1.972, 2.026, 1.868, 1.949, 2.033, 1.887, 1.856, 2.112, 2.117, 2.129, 1.994, 2.03, 1.925, 1.916, 1.949, 1.956, 2.119, 1.894, 2.034, 1.766, 1.994, 1.985, 2.068, 1.796, 2.172, 2.09, 2.017, 2.152, 2.238, 2.027, 2.214, 2.159, 2.098, 1.963, 2.063, 2.292, 2.21, 1.84, 2.102, 2.1, 2.086, 2.008, 2.048, 1.978, 1.87, 2.16, 1.977, 2.335, 2.053, 2.084, 2.056, 2.005, 2.161, 2.088, 2.161, 2.078, 2.038, 2.117, 2.148, 2.043, 2.342, 2.142, 2.306, 2.043, 1.881, 1.918, 2.252)

# UI
ui <- fluidPage(
  titlePanel("EPOC-Modellfunktion"),
  fluidRow(
    column(3,
     style = "height: 90vh; overflow-y: auto;",
     
      tags$h4(tags$strong("Modellparameter:")),
      sliderInput("VO2", "V̇O2 Amplitude", min = 0.0, max = 7.0, value = 1.7, step = 0.001),
      sliderInput("Tau", "Tau", min = 5, max = 360, value = 23, step = 1.0),
      sliderInput("VO2_Start", "V̇O2 Start", min = 0.0, max = 4.0, value = 0.4, step = 0.001),
      sliderInput("VO2_Ruhe", "V̇O2 Ruhe", min = 0, max = 1, value = 0.3, step = 0.001),
      sliderInput("time_delay", "Zeitverzögerung", min = 0, max = 300, value = 11, step = 1),
      conditionalPanel(
        condition = "output.showFitSlider",
        sliderInput("nlsLM_end", "nlsLM Ende", min = 10, max = max(t_data), value = max(t_data), step = 1)
      ),
      actionButton("toggle_data", "Beispieldaten einfügen"),
      br(), br(),
      fileInput("file_upload", "CSV-Datei hochladen", accept = ".csv"),
      tags$h4(tags$strong("Modelanpassung:")),
      actionButton("optimize", "nlsLM - Fit"),
      br(), br(),
      tags$h4(tags$strong("Berechnung - Ruhesauerstoffaufnahme:")),
      radioButtons("geschlecht", "Geschlecht:", choices = c("Männlich", "Weiblich")),
      sliderInput("koerpermasse", "Körpermasse [kg]:", min = 40, max = 150, value = 55),
      sliderInput("koerperlaenge", "Körperlänge [cm]:", min = 140, max = 220, value = 160),
      sliderInput("alter", "Alter [Jahre]:", min = 18, max = 100, value = 24),
      sliderInput("rq", "RQ:", min = 0.7, max = 1.0, value = 0.77, step = 0.01),
      actionButton("berechne_vo2_ruhe", "VO2 Ruhe berechnen")
    ),
    mainPanel(
      width = 9,
      plotlyOutput("plot"),
      uiOutput("instructions")
    )
  )
)

# Server
server <- function(input, output, session) {
  show_data <- reactiveVal(FALSE)
  uploaded_data <- reactiveVal(NULL)
  
  observeEvent(input$toggle_data, {
    show_data(!show_data())
    max_t <- max(t_data)
    updateSliderInput(session, "nlsLM_end", min = input$time_delay, max = max_t, value = max_t)
  })
  
  observeEvent(input$file_upload, {
    req(input$file_upload)
    df <- read.csv(input$file_upload$datapath)
    if ("t_s" %in% names(df) && "VO2_t" %in% names(df)) {
      uploaded_data(df)
      show_data(TRUE)
      max_t <- max(df$t_s)
      updateSliderInput(session, "nlsLM_end", min = input$time_delay, max = max_t, value = max_t)
    } else {
      showModal(modalDialog(
        title = "Invalid CSV",
        "Die csv-Datei muss 't_s' und 'VO2_t' als Spaltennamen beinhalten.",
        easyClose = TRUE,
        footer = NULL
      ))
    }
  })
  
  # Neue Funktionen zur Berechnung des Grundumsatzes und RMR
  berechne_grundumsatz <- function(geschlecht, masse, laenge, alter) {
    if (geschlecht == "Männlich") {
      return(66.5 + (13.75 * masse) + (5.003 * laenge) - (6.775 * alter))
    } else {
      return(655.1 + (9.563 * masse) + (1.850 * laenge) - (4.676 * alter))
    }
  }
  
  berechne_rmr <- function(grundumsatz, rq, geschlecht) {
    ka <- 19.946  # Annahme für RQ = 0.77, passen Sie dies an, wenn nötig
    faktor <- if(geschlecht == "Männlich") 1.287 else 1.278
    return((grundumsatz / (24 * 60 * ka)) * 4.1868 * faktor)
  }
  
  # Reaktion auf den Klick des "VO2 Ruhe berechnen" Buttons
  observeEvent(input$berechne_vo2_ruhe, {
    grundumsatz <- berechne_grundumsatz(
      input$geschlecht,
      input$koerpermasse,
      input$koerperlaenge,
      input$alter
    )
    
    rmr <- berechne_rmr(grundumsatz, input$rq, input$geschlecht)
    
    # Aktualisieren Sie den VO2_Ruhe Slider mit dem berechneten Wert
    updateSliderInput(session, "VO2_Ruhe", value = round(rmr, 3))
    
    # Zeigen Sie eine Benachrichtigung mit dem berechneten Wert an
    showNotification(paste("Berechnete Ruhesauerstoffaufnahme:", round(rmr, 3), "l/min"), type = "message")
  })
  
  
  
  observe({
    if (input$VO2_Start < input$VO2_Ruhe) {
      updateSliderInput(session, "VO2_Start", value = input$VO2_Ruhe)
    }
  })
  
  output$showFitSlider <- reactive({
    show_data()
  })
  outputOptions(output, "showFitSlider", suspendWhenHidden = FALSE)
  
  observeEvent(input$optimize, {
    if (show_data()) {
      Beispieldaten <- if (is.null(uploaded_data())) {
        data.frame(t_s = t_data, VO2_t = VO2_data)
      } else {
        uploaded_data()
      }
      
      t_delay <- isolate(input$time_delay)
      nlsLM_end <- isolate(input$nlsLM_end)
      
      # Finde den nächstgelegenen Zeitpunkt zu t_delay
      closest_index <- which.min(abs(Beispieldaten$t_s - t_delay))
      
      # Setze VO2_Start auf den gerundeten Wert des nächsten Punktes
      VO2_Start <- round(Beispieldaten$VO2_t[closest_index], 3)
      
      updateSliderInput(session, "VO2_Start", value = VO2_Start)
      
      data_subset <- Beispieldaten[Beispieldaten$t_s >= t_delay & Beispieldaten$t_s <= nlsLM_end, ]
      
      start_values <- list(VO2 = isolate(input$VO2), Tau = isolate(input$Tau))
      
      fit <- try(nlsLM(VO2_t ~ VO2 * (1 - exp(-(t_s - t_delay) / Tau)) + VO2_Start,
                       data = data_subset,
                       start = start_values,
                       lower = c(VO2 = 0.5, Tau = 5),
                       upper = c(VO2 = 7.0, Tau = 360),
                       control = nls.control(maxiter = 1024)), silent = TRUE)
      
      if (!inherits(fit, "try-error")) {
        params <- as.list(coef(fit))
        updateSliderInput(session, "VO2", value = params$VO2)
        updateSliderInput(session, "Tau", value = params$Tau)
      } else {
        showModal(modalDialog(
          title = "nlsLM fehlgeschlagen",
          "Die nicht-lineare Regression mittels Levenberg-Marquardt-Algorithmus konnte nicht erfolgreich durchgeführt werden.",
          easyClose = TRUE,
          footer = NULL
        ))
      }
    }
  })
  
  output$plot <- renderPlotly({
    VO2 <- input$VO2
    Tau <- input$Tau
    VO2_Start <- input$VO2_Start
    t_delay <- input$time_delay
    VO2_Ruhe <- input$VO2_Ruhe
    nlsLM_end <- input$nlsLM_end
    
    model_function <- function(t_s, VO2, Tau, VO2_Start, t_delay) {
      VO2 * (1 - exp(-(t_s - t_delay) / Tau)) + VO2_Start
    }
    
    Beispieldaten <- if (show_data() && !is.null(uploaded_data())) {
      uploaded_data()
    } else {
      data.frame(t_s = t_data, VO2_t = VO2_data)
    }
    
    tau4 <- 4 * Tau
    max_t <- max(Beispieldaten$t_s, na.rm = TRUE)
    max_x <- max(1.05 * tau4 + t_delay, max_t * 1.05)
    t_s <- seq(0, max_x * 1.20, by = 1)
    model_values <- ifelse(t_s >= t_delay, model_function(t_s, VO2, Tau, VO2_Start, t_delay), NA)
    VO2_SS_gross <- model_function(tau4 + t_delay, VO2, Tau, VO2_Start, t_delay)
    VO2_SS_net <- VO2_SS_gross - VO2_Ruhe
    max_y <- max(1.05 * VO2_SS_gross, max(Beispieldaten$VO2_t, na.rm = TRUE) * 1.05)
    
    eq_text <- sprintf("V̇O₂ = %.3f * (1 - e<sup>-(t - %.2f) / %.2f</sup>) + %.3f",
                       VO2, t_delay, Tau, VO2_Start)
    VO2_SS_gross_text <- sprintf("V̇O₂ <sub>SS,gross</sub>: %.3f", VO2_SS_gross)
    VO2_SS_net_text <- sprintf("V̇O₂ <sub>SS,net</sub>: %.3f", VO2_SS_net)
    t_halb <- Tau * log(2)
    
    shapes <- list(
      list(
        type = "line", x0 = t_delay, x1 = t_delay, y0 = 0, y1 = max_y * 1.2,
        line = list(color = "gray", width = 1, dash = "dash")
      ),
      list(
        type = "line", x0 = Tau + t_delay, x1 = Tau + t_delay, y0 = 0, y1 = max_y * 1.2,
        line = list(color = "gray", width = 1, dash = "dash")
      ),
      list(
        type = "line", x0 = tau4 + t_delay, x1 = tau4 + t_delay, y0 = 0, y1 = max_y * 1.2,
        line = list(color = "gray", width = 1, dash = "dash")
      ),
      if (show_data()) {
        list(
          type = "line", x0 = nlsLM_end, x1 = nlsLM_end, y0 = 0, y1 = max_y * 1.2,
          line = list(color = "gray", width = 1, dash = "dash")
        )
      },
      list(
        type = "line", x0 = 0, x1 = max_x, y0 = VO2_Ruhe, y1 = VO2_Ruhe,
        line = list(color = '#1CADE4', width = 1, dash = "solid")
      )
    )
    
    p <- plot_ly() %>%
      add_trace(x = ~t_s, y = ~model_values, type = 'scatter', mode = 'lines',
                name = 'Modellfunktion', line = list(color = '#EF6F6A')) %>%
      layout(title = "V̇O<sub>2</sub>-Modellfunktion",
             margin = list(t = 40),
             xaxis = list(title = "t [s]", range = c(0, max_x)),
             yaxis = list(title = "V̇O<sub>2</sub> [l·min<sup>-1</sup>]", tickformat = ".3f"),
             shapes = shapes,
             annotations = list(
               list(
                 x = max_x * 0.70,
                 y = max_y * 0.6,
                 text = eq_text,
                 showarrow = FALSE,
                 xanchor = 'left',
                 yanchor = 'bottom',
                 font = list(
                   family = "Arial, sans-serif",
                   size = 12,
                   color = "black"
                 )
               ),
               list(
                 x = max_x * 0.70,
                 y = max_y * 0.5,
                 text = VO2_SS_gross_text,
                 showarrow = FALSE,
                 xanchor = 'left',
                 yanchor = 'bottom',
                 font = list(
                   family = "Arial, sans-serif",
                   size = 12,
                   color = "black"
                 )
               ),
               list(
                 x = max_x * 0.70,
                 y = max_y * 0.4,
                 text = VO2_SS_net_text,
                 showarrow = FALSE,
                 xanchor = 'left',
                 yanchor = 'bottom',
                 font = list(
                   family = "Arial, sans-serif",
                   size = 12,
                   color = "black"
                 )
               ),
               list(
                 x = max_x * 0.70,
                 y = max_y * 0.3,
                 text = paste("T<sub>1/2</sub>:", round(t_halb, 1)),
                 showarrow = FALSE,
                 xanchor = 'left',
                 yanchor = 'bottom',
                 font = list(
                   family = "Arial, sans-serif",
                   size = 12,
                   color = "black"
                 )
               ),
               list(
                 x = t_delay, y = max_y * 0.9, text = sprintf("t<sub>delay</sub>: %.1f", t_delay), showarrow = FALSE, xanchor = "left", yanchor = "bottom",
                 textangle = -90, font = list(size = 11)
               ),
               list(
                 x = Tau + t_delay, y = max_y * 0.9, text = sprintf("tau: %.1f", Tau), showarrow = FALSE, xanchor = "left", yanchor = "bottom",
                 textangle = -90, font = list(size = 11)
               ),
               list(
                 x = tau4 + t_delay, y = max_y * 0.9, text = sprintf("4tau: %.1f", tau4), showarrow = FALSE, xanchor = "left", yanchor = "bottom",
                 textangle = -90, font = list(size = 11)
               ),
               if (show_data()) {
                 list(
                   x = nlsLM_end, y = max_y * 0.9, text = sprintf("nlsLM Ende: %.1f", nlsLM_end), showarrow = FALSE, xanchor = "left", yanchor = "bottom",
                   textangle = -90, font = list(size = 11)
                 )
               }
             )) %>%
      add_trace(x = ~t_s, y = ~rep(VO2_Ruhe, length(t_s)), type = 'scatter', mode = 'lines',
                name = 'V̇O<sub>2, Ruhe</sub>', line = list(color = '#1CADE4'))
    
    if (show_data()) {
      model_values_at_data <- model_function(Beispieldaten$t_s, VO2, Tau, VO2_Start, t_delay)
      
      data_subset <- Beispieldaten[Beispieldaten$t_s >= t_delay & Beispieldaten$t_s <= input$nlsLM_end, ]
      
      ss_res <- sum((data_subset$VO2_t - model_values_at_data[Beispieldaten$t_s >= t_delay & Beispieldaten$t_s <= input$nlsLM_end])^2, na.rm = TRUE)
      ss_tot <- sum((data_subset$VO2_t - mean(data_subset$VO2_t, na.rm = TRUE))^2, na.rm = TRUE)
      r_squared <- 1 - (ss_res / ss_tot)
      
      p <- p %>%
        add_trace(data = Beispieldaten, x = ~t_s, y = ~VO2_t, type = 'scatter', mode = 'markers+lines',
                  name = 'V̇O<sub>2</sub>', 
                  marker = list(color = 'rgba(38, 131, 198, 0.9)', size = 5.0),
                  line = list(color = 'rgba(38, 131, 198, 1.0)', width = 0.65, dash = '4 4')) %>%
        layout(annotations = list(
          list(
            x = max_x * 0.70,
            y = max_y * 0.2,
            text = sprintf("R²: %.3f", r_squared),
            showarrow = FALSE,
            xanchor = 'left',
            yanchor = 'bottom',
            font = list(
              family = "Arial, sans-serif",
              size = 12,
              color = "black"
            )
          )
        ))
    }
    
    p
  })
  output$instructions <- renderUI({
    HTML(
      "<div style='margin-top: 20px; padding: 10px; background-color: #f0f0f0; border: 1px solid #ddd; border-radius: 5px; width: fit-content;'>
        <h4 style='color: #333;'><strong>Anleitung - Modellanpassung:</strong></h4>
        <ol style='color: #555; list-style-position: outside; padding-left: 20px;'>
          <li>Beispiel VO2-Daten oder eigene VO2-Daten als CSV-Datei einfügen.</li>
          <li>Die Zeitverzögerung festlegen, um den Startpunkt der Modellanpassung zu bestimmen, und bei Bedarf das Ende der Anpassung anpassen.</li>
          <li>Modellanpassung 'Fit: nlsLM' durchführen.</li>
          <li>Alternativ: Manuelle Modellanpassung der Modellparameter mit den Schiebereglern.</li>
        </ol>
        <div style='margin-top: 20px;'></div>
        <pre style='background-color: #f8f8f8; padding: 10px; border: 1px solid #ddd; border-radius: 5px; width: fit-content;'>
VO2-Daten können als CSV-Datei im folgenden Format hochgeladen werden:
t_s,VO2_t
0.0,0.479
1.0,0.459
2.3,0.488
…
        </pre>
      </div>"
    )
  })
}

# App ausführen
shinyApp(ui = ui, server = server)



```

::: {.callout-note icon=false collapse=true appearance="simple"}
### Modellanpassung - Details:

**nlsLM - Fit:**

- Verwendete Funktion: nlsLM
- Parameter: V̇O2 (Amplitude), Tau
- Modell: V̇O2_t ~ V̇O2 · (1 - exp(-(t_s - t_delay) / Tau)) + V̇O2_Start
- Grenzen:
  - V̇O2: *0.5 bis 7.0 l/min*
  - Tau: *5 bis 360 s*
- Prozess:
  a) Bestimmung des V̇O2_Start-Wertes:
     - Finden des nächstgelegenen Zeitpunkts zu t_delay in den Daten
     - Setzen von V̇O2_Start auf den gerundeten V̇O2-Wert dieses Punktes
  b) Datenauswahl:
     - Verwendung der Daten zwischen *t_delay* und *nlsLM_end*
  c) Startwertebestimmung:
     - Verwendung der aktuellen Slider-Werte für V̇O2 und Tau
  d) Durchführung der nicht-linearen Regression:
     - Verwendung des Levenberg-Marquardt-Algorithmus
     - Maximale Iterationen: *1024*
- Optimierung: Minimierung der Residuenquadratsumme (RSS)
  
:::

## Einteilung der VO2-Kinetik in drei Phasen

Die Analyse der $\dot{V}O_{2}$-Kinetik kann durch die Berücksichtigung von Zeitverzögerungen und/oder Exponentialfunktionen zweiter Ordnung verfeinert werden. Dazu kann die Anpassungsreaktion des $\dot{V}O_{2}$ nach Beginn der sportlichen Betätigung in drei Phasen unterteilt werden, wie in @fig-3_Phasen zu sehen [@Whipp1982]. Dies ermöglicht es, die erhöhte Komplexität der $\dot{V}O_{2}$-Kinetik oberhalb der LT besser darzustellen und eine präzisere Analyse der physiologischen Anpassungen bei körperlicher Belastung durchzuführen, da oberhalb der LT der $\dot{V}O_2$-Umsatz über 10 ml O~2~·W^-1^·min^-1^  steigt [@Barstow1994; @Bearden2000; @Siestema2020]. Die 3 Phasen lassen sich wie folgt beschreiben:

::: columns
::: {.column width="72%"}
![Beispiel der Drei-Phasen der VO2-Kinetik bis zum Erreichen eines SS nach Beginn der Belastung (McNulty & Robergs, 2017)](McNulty2017_VO2_3_Phase.png){#fig-3_Phasen height="500"} 
:::
::: {.column width="2%"}
:::
::: {.column width="25%"}
:::
:::






::: columns
::: {.column width="59%"}
**Phase 1:**<br>
Phase I, häufig als kardiodynamische Phase bezeichnet, umfasst ungefähr die ersten 10 bis 25 Sekunden nach Beginn der Belastung. Aufgrund des schnellen Anstiegs des Herzzeitvolumens und des Lungenblutflusses zu Beginn der Belastung steigt $\dot{V}O_2$ abrupt an. In dieser Phase bleibt die Zusammensetzung des gemischtvenösen Blutes sowie der respiratorische Quotient ($RQ = \frac{\dot{V}CO_{2}}{\dot{V}O_{2}}$) im Vergleich zum Ruhezustand unverändert. Somit ist der Anstieg des $\dot{V}O_2$ ausschließlich auf das erhöhte Herzminutenvolumen zurückzuführen, da sich die Sauerstoffkonzentration im arteriellen und venösen Blut (C~a~ und C~v~) noch nicht verändert haben. Dies lässt sich mit Hilfe der Fick'schen Gleichung @eq-FICK verdeutlichen [@Fick1855; @Whipp1982; @Stirling2005; @Clark2014; @Siestema2020].

$$
\dot{V}O_2 = \dot{Q} \cdot (C_a - C_v) 
$$ {#eq-FICK}

Da die anfängliche „kardiodynamische“ Phase der $\dot{V}O_2$-Reaktion nicht direkt den Sauerstoffumsatz der aktiven Muskulatur repräsentiert, werden in der Regel die ersten 20 bis 25 Sekunden von Phase I bei Modellierungen der $\dot{V}O_2$-Kinetik aus der Modellanpassung ausgeschlossen [@Bearden2000; @Oezyener2001].

**Phase 2:**<br>
Phase 2 folgt auf Phase 1 und ist durch einen schnellen, monoexponentiellen Anstieg des $\dot{V}O_2$ gekennzeichnet. Die Zeitkonstante beträgt im Durchschnitt etwa 30 [@Siestema2020] bis 33 Sekunden [@Oezyner2001]. Anschließend wird entweder ein SS erreicht oder der $\dot{V}O_2$ steigt weiter an, was dann in dem Modell als Phase 3 bezeichnet wird [@Stirling2005].

**Phase 3:**<br>
In Phase 3 des $\dot{V}O_2$-Modells wird bei leichten bis moderaten Belastungen unterhalb der Laktatschwelle (LT) nach etwa 3 Minuten ein stabiler Zustand (SS) erreicht. Dieses Verhalten beschreibt, wie der $\dot{V}O_2$ sich im 3-Phasen-Modell für Belastungen unterhalb der LT verhält.<br>
Bei Belastungen oberhalb der LT erreicht der $\dot{V}O_2$ nach der primären Anpassungsreaktion keinen SS mehr, sondern steigt weiter an (siehe @fig-Verlauf_ON). Diese zusätzliche Komponente, meist als "slow component" bezeichnet und überlagert die anfängliche monoexponentielle Funktion aus Phase 2. 
:::
::: {.column width="2%"}
:::
::: {.column width="39%"}
![Beispiele der VO2-Anstiegskinetik bei einem Probanden für unterschiedliche Intensitäten: sehr intensiv, schwer, mittelschwer und moderat (von oben nach unten). Unterhalb der VO2-Daten ist der zugehörige Residuenplot zu sehen. In Spalte A ist das mono-exponentielle Modell und in Spalte B das doppel-exponentielle Modell dargestellt (Özyener 2001).](Ozyener_ON.png){#fig-Verlauf_ON}
::: 
:::
Mathematische Modellierungen haben gezeigt, dass die langsame Komponente etwa 90 bis 200 Sekunden nach Beginn des Übergangs einsetzt [@Whipp1972; @Barstow1991; @PatersonWhipp1991; @Whipp1994; @Bearden2000; @Siestema2020]. Die Komponente der Anpassungsreaktion bis zum Beginn der slow component wird in der Literatur oft als "primary component" oder "fundamental component" bezeichnet [@Barstow1994; @Oeyzener2001]. Im Folgenden wird diese Hauptkomponente mit dem Index "fast" gekennzeichnet.





## Biexponentielle Modellfunktion des Sauerstoffvolumenstroms in Abhängigkeit von der Zeit während der Belastung

Phase 2 und 3 des 3-Phasen-Modells lassen sich mit Hilfe der @eq-VO2_Start2 modellieren:  

$$
\dot{V}O_2(t) = \dot{V}O_{2,\text{fast}} \cdot \left(1 - e^{-\frac{t_s - t_\text{delay, fast}}{\tau_\text{fast}}}\right) + 
\dot{V}O_{2,\text{slow}} \cdot \left(1 - e^{-\frac{t_s - t_\text{delay,slow}}{\tau_\text{slow}}}\right) + 
\dot{V}O_{2,\text{Start}}
$${#eq-VO2_Start2}

$\dot{V}O_{2,\text{fast}}$ repräsentiert die Amplitude der Hauptkomponente, während $\dot{V}O_{2,\text{slow}}$ die Amplitude der slow component beschreibt. Die Zeitkonstanten $\tau_\text{fast}$ und $\tau_\text{slow}$ steuern die Geschwindigkeit der jeweiligen Reaktionsprozesse, wobei die Zeitverzögerungen $t_\text{delay, fast}$ und $t_\text{delay, slow}$ den Beginn dieser Prozesse bestimmen. $\dot{V}O_{2,\text{Start}}$ beschreibt den Sauerstoffvolumenstrom zu Beginn der Belastung.

In der folgenden Shiny-App wird die Modellanpassung der Gleichung @eq-VO2_Start2 an Beispieldaten demonstriert. Eigene $\dot{V}O_{2}$-Daten können hochgeladen und mittels nichtlinearer Regression an die Modellparameter angepasst werden.

## Bi-exponentielle Shiny-App
```{shinylive-r}
#| standalone: true
#| viewerHeight: 900

library(shiny)
library(plotly)
library(minpack.lm)
library(dplyr)
library(shinyjs)

# Beispieldaten
t_data <- c(0.0, 2.7, 5.3, 8.3, 11.5, 13.6, 15.4, 17.1, 19.8, 21.1, 22.3, 23.5, 24.9, 27.5, 30.9, 32.4, 35.0, 38.3, 39.7, 41.6, 43.7, 45.3, 46.8, 49.8, 52.0, 53.6, 55.5, 57.5, 59.0, 60.7, 62.4, 64.4, 66.6, 68.1, 69.8, 71.4, 73.3, 75.0, 76.7, 78.4, 80.0, 82.2, 83.9, 85.4, 87.3, 88.8, 90.6, 92.0, 93.6, 94.8, 96.3, 97.9, 99.5, 101.1, 102.7, 104.3, 106.0, 107.7, 109.3, 110.9, 112.5, 114.8, 116.4, 118.1, 119.7, 121.3, 122.8, 124.3, 125.9, 127.4, 128.9, 130.6, 132.1, 133.6, 135.6, 137.1, 138.6, 140.1, 141.6, 143.1, 144.4, 145.8, 147.2, 148.4, 149.9, 151.6, 153.1, 154.7, 156.1, 157.4, 158.7, 160.5, 161.9, 163.3, 164.7, 166.2, 167.6, 169.1, 172.6, 173.9, 175.4, 176.7, 178.1, 179.6, 181.0, 182.4, 183.8, 185.3, 186.7, 188.2, 189.6, 191.0, 192.4, 193.8, 195.2, 196.6, 198.0, 199.3, 200.7, 202.0, 203.3, 204.7, 206.1, 207.5, 208.8, 210.2, 211.6, 213.0, 214.3, 215.7, 217.1, 218.4, 219.8, 221.1, 222.4, 223.7, 225.0, 226.3, 227.7, 229.0, 230.5, 231.9, 233.1, 234.4)
VO2_data <- c(0.823, 0.898, 0.966, 1.122, 1.281, 1.313, 1.463, 1.537, 1.602, 1.595, 1.598, 1.674, 1.923, 2.168, 2.439, 2.670, 2.797, 2.712, 2.688, 2.968, 3.228, 3.371, 3.673, 3.938, 4.076, 3.959, 3.886, 3.880, 3.928, 3.962, 4.173, 4.381, 4.268, 4.208, 4.234, 4.132, 4.109, 4.192, 4.304, 4.493, 4.352, 4.349, 4.332, 4.346, 4.345, 4.373, 4.320, 4.286, 4.353, 4.346, 4.287, 4.322, 4.250, 4.332, 4.257, 4.161, 4.120, 4.144, 4.115, 4.158, 4.186, 4.206, 4.220, 4.259, 4.291, 4.279, 4.295, 4.339, 4.371, 4.425, 4.448, 4.474, 4.517, 4.536, 4.544, 4.528, 4.432, 4.371, 4.367, 4.430, 4.492, 4.535, 4.510, 4.433, 4.399, 4.305, 4.308, 4.304, 4.366, 4.434, 4.501, 4.535, 4.512, 4.463, 4.509, 4.529, 4.586, 4.624, 4.740, 4.769, 4.788, 4.794, 4.824, 4.830, 4.888, 4.887, 4.915, 4.924, 4.955, 4.965, 4.982, 4.992, 5.016, 5.024, 5.033, 5.066, 5.081, 5.053, 5.035, 4.997, 5.018, 5.038, 5.035, 5.039, 4.970, 4.970, 4.973, 4.964, 4.945, 4.953, 4.974, 4.984, 4.997, 4.995, 4.946, 4.997, 5.014, 5.092, 5.072, 5.111, 5.088, 5.065, 5.061, 5.038)

# UI
ui <- fluidPage(
  useShinyjs(),
  titlePanel("Biexponentielle V̇O2-Modellfunktion"),
  fluidRow(
    column(3,
           style = "height: 90vh; overflow-y: auto;",
           
           tags$h4(tags$strong("Modellparameter:")),
           sliderInput("VO2", "V̇O2_fast", min = 0.0, max = 7.0, value = 3.7, step = 0.01),
           sliderInput("tau_fast", "Tau_fast", min = 10, max = 120, value = 45, step = 0.1),
           sliderInput("VO2_Start", "V̇O2_Start", min = 0.0, max = 4.0, value = 0.8, step = 0.01),
           sliderInput("VO2_Ruhe", "V̇O2_Ruhe", min = 0, max = 1, value = 0.4, step = 0.1),
           sliderInput("time_delay", "Zeitverzögerung_fast", min = 0, max = 600, value = 0, step = 1),
           sliderInput("VO2_slow", "V̇O2_slow", min = 0.0, max = 1.0, value = 0.3, step = 0.01),
           sliderInput("tau_slow", "Tau_slow", min = 90, max = 600, value = 110, step = 0.1),
           sliderInput("time_delay_slow", "Zeitverzögerung_slow", min = 0, max = 1200, value = 160, step = 1),
           conditionalPanel(
             condition = "output.showFitSlider",
             sliderInput("nlsLM_end", "nlsLM Ende", min = 10, max = 300, value = 300, step = 1)
           ),
           actionButton("toggle_data", "Beispieldaten einfügen"),
           br(), br(),
           fileInput("file_upload", "CSV-Datei hochladen", accept = ".csv"),
           tags$h4(tags$strong("Modellanpassung:")),
           actionButton("fit_fast", "1. Fit: Schnelle Komponente"),br(),
           actionButton("fit_slow", "2. Fit: Langsame Komponente & Verfeinerung"),br(),
           actionButton("fit_optimize", "3. Fit: Optimierung")
    ),
    mainPanel(
      width = 9,
      plotlyOutput("plot"),
      uiOutput("instructions")
    )
  )
)

# Server
server <- function(input, output, session) {
  show_data <- reactiveVal(FALSE)
  uploaded_data <- reactiveVal(NULL)
  current_data <- reactiveVal(NULL)
  current_r_squared <- reactiveVal(NA)
  
  calculate_r_squared <- function(observed, predicted) {
    ss_res <- sum((observed - predicted)^2, na.rm = TRUE)
    ss_tot <- sum((observed - mean(observed, na.rm = TRUE))^2, na.rm = TRUE)
    return(1 - (ss_res / ss_tot))
  }
  
  observeEvent(input$toggle_data, {
    show_data(!show_data())
    if (show_data()) {
      current_data(data.frame(t_s = t_data, VO2_t = VO2_data))
    } else {
      current_data(NULL)
    }
  })
  
  observeEvent(input$file_upload, {
    req(input$file_upload)
    df <- read.csv(input$file_upload$datapath)
    if ("t_s" %in% names(df) && "VO2_t" %in% names(df)) {
      uploaded_data(df)
      current_data(df)
      show_data(TRUE)
    } else {
      showModal(modalDialog(
        title = "Invalid CSV",
        "Die csv-Datei muss 't_s' und 'VO2_t' als Spaltennamen beinhalten.",
        easyClose = TRUE,
        footer = NULL
      ))
    }
  })
  
  observe({
    req(input$VO2_Start, input$VO2_Ruhe)
    if (input$VO2_Start < input$VO2_Ruhe) {
      updateSliderInput(session, "VO2_Start", value = input$VO2_Ruhe)
    }
  })
  
  # Hilfsfunktion zur Bestimmung des nächstgelegenen VO2-Wertes
  find_nearest_VO2 <- function(data, t_delay) {
    req(data, t_delay)
    if (nrow(data) == 0) return(NA)
    
    index <- which.min(abs(data$t_s - t_delay))
    return(data$VO2_t[index])
  }
  
  
  observeEvent(input$fit_fast, {
    req(current_data())
    Beispieldaten <- current_data()
    
    t_delay <- isolate(input$time_delay)
    
    # Bestimme den nächstgelegenen VO2-Wert als VO2_Start
    VO2_Start <- find_nearest_VO2(Beispieldaten, t_delay)
    
    # Aktualisiere den VO2_Start Slider
    updateSliderInput(session, "VO2_Start", value = VO2_Start)
    
    data_subset <- Beispieldaten[Beispieldaten$t_s >= t_delay, ]
    
    best_fit <- NULL
    best_rss <- Inf
    
    withProgress(message = 'Fitting in progress', value = 0, {
      for (i in 1:100) {
        # Randomisierte Startwerte innerhalb der Slider-Grenzen
        VO2_start <- runif(1, 0.5, 7.0)
        tau_fast_start <- runif(1, 10, 120)
        
        start_values <- list(
          VO2 = VO2_start,
          tau_fast = tau_fast_start
        )
        
        fit <- try(nlsLM(VO2_t ~ VO2 * (1 - exp(-(t_s - t_delay) / tau_fast)) + VO2_Start,
                         data = data_subset,
                         start = start_values,
                         lower = c(VO2 = 0.5, tau_fast = 10),
                         upper = c(VO2 = 7.0, tau_fast = 120),
                         control = nls.control(maxiter = 1024)), silent = TRUE)
        
        if (!inherits(fit, "try-error")) {
          rss <- sum(residuals(fit)^2)
          if (rss < best_rss) {
            best_rss <- rss
            best_fit <- fit
          }
        }
        
        incProgress(1/100, detail = paste("Iteration", i))
      }
    })
    
    if (!is.null(best_fit)) {
      params <- as.list(coef(best_fit))
      updateSliderInput(session, "VO2", value = params$VO2)
      updateSliderInput(session, "tau_fast", value = params$tau_fast)
      updateSliderInput(session, "VO2_slow", value = 0)
      updateSliderInput(session, "tau_slow", value = 0)
      
      predicted <- VO2_Start + params$VO2 * (1 - exp(-(data_subset$t_s - t_delay) / params$tau_fast))
      r_squared <- calculate_r_squared(data_subset$VO2_t, predicted)
      current_r_squared(r_squared)
      
      showNotification(sprintf("Schritt 1 erfolgreich. R²: %.3f", r_squared), type = "message")
    } else {
      showModal(modalDialog(
        title = "nlsLM fehlgeschlagen",
        "Die nicht-lineare Regression mittels Levenberg-Marquardt-Algorithmus konnte nicht erfolgreich durchgeführt werden.",
        easyClose = TRUE,
        footer = NULL
      ))
    }
  })
  
  observeEvent(input$fit_slow, {
    req(current_data())
    Beispieldaten <- current_data()
    
    req(input$time_delay, input$VO2_Start, input$VO2, input$tau_fast)
    
    t_delay <- input$time_delay
    VO2_Start <- input$VO2_Start
    VO2 <- input$VO2
    tau_fast <- input$tau_fast
    
    # Setze VO2_slow und tau_slow auf die initialen Werte
    VO2_slow <- 0.4
    tau_slow <- 160
    
    data_subset <- Beispieldaten[Beispieldaten$t_s >= t_delay, ]
    
    # Berechne die obere Grenze für t_delay_slow
    t_delay_slow_upper <- max(Beispieldaten$t_s)
    
    best_fit <- NULL
    best_rss <- Inf
    best_t_delay_slow <- NULL
    best_VO2_fast <- VO2
    best_tau_fast <- tau_fast
    best_VO2_slow <- VO2_slow
    best_tau_slow <- tau_slow
    
    # Berechne die Anzahl der Iterationen für t_delay_slow
    num_iterations <- floor(t_delay_slow_upper - (tau_fast + t_delay)) + 1
    
    withProgress(message = 'Fitting in progress', value = 0, {
      for (i in 1:num_iterations) {
        t_delay_slow <- tau_fast + t_delay + i - 1
        
        # Fit VO2_fast und tau_fast nach unten
        current_VO2_fast <- best_VO2_fast
        current_tau_fast <- best_tau_fast
        current_VO2_slow <- best_VO2_slow
        current_tau_slow <- best_tau_slow
        
        model <- function(VO2_fast, tau_fast, VO2_slow, tau_slow, t_delay_slow) {
          VO2_fast * (1 - exp(-(data_subset$t_s - t_delay) / tau_fast)) + 
            VO2_slow * (1 - exp(-(pmax(data_subset$t_s - t_delay_slow, 0)) / tau_slow)) + 
            VO2_Start
        }
        
        current_rss <- sum((data_subset$VO2_t - model(current_VO2_fast, current_tau_fast, current_VO2_slow, current_tau_slow, t_delay_slow))^2)
        
        # Optimiere VO2_fast und tau_fast
        while (TRUE) {
          improved <- FALSE
          
          # Versuche VO2_fast zu reduzieren
          if (current_VO2_fast > 0.1) {
            new_VO2_fast <- max(0.1, current_VO2_fast - 0.01)
            new_rss <- sum((data_subset$VO2_t - model(new_VO2_fast, current_tau_fast, current_VO2_slow, current_tau_slow, t_delay_slow))^2)
            if (new_rss < current_rss) {
              current_VO2_fast <- new_VO2_fast
              current_rss <- new_rss
              improved <- TRUE
            }
          }
          
          # Versuche tau_fast zu reduzieren
          if (current_tau_fast > 10) {
            new_tau_fast <- max(10, current_tau_fast - 0.1)
            new_rss <- sum((data_subset$VO2_t - model(current_VO2_fast, new_tau_fast, current_VO2_slow, current_tau_slow, t_delay_slow))^2)
            if (new_rss < current_rss) {
              current_tau_fast <- new_tau_fast
              current_rss <- new_rss
              improved <- TRUE
            }
          }
          
          if (!improved) break
        }
        
        # Optimiere VO2_slow und tau_slow
        # Zuerst nach unten
        while (TRUE) {
          improved <- FALSE
          
          # Versuche VO2_slow zu reduzieren
          if (current_VO2_slow > 0.05) {
            new_VO2_slow <- max(0.05, current_VO2_slow - 0.01)
            new_rss <- sum((data_subset$VO2_t - model(current_VO2_fast, current_tau_fast, new_VO2_slow, current_tau_slow, t_delay_slow))^2)
            if (new_rss < current_rss) {
              current_VO2_slow <- new_VO2_slow
              current_rss <- new_rss
              improved <- TRUE
            }
          }
          
          # Versuche tau_slow zu reduzieren
          if (current_tau_slow > 90) {
            new_tau_slow <- max(90, current_tau_slow - 0.1)
            new_rss <- sum((data_subset$VO2_t - model(current_VO2_fast, current_tau_fast, current_VO2_slow, new_tau_slow, t_delay_slow))^2)
            if (new_rss < current_rss) {
              current_tau_slow <- new_tau_slow
              current_rss <- new_rss
              improved <- TRUE
            }
          }
          
          if (!improved) break
        }
        
        # Dann nach oben
        while (TRUE) {
          improved <- FALSE
          
          # Versuche VO2_slow zu erhöhen
          if (current_VO2_slow < 1.0) {  # Angenommen, 1.0 ist die obere Grenze für VO2_slow
            new_VO2_slow <- min(1.0, current_VO2_slow + 0.01)
            new_rss <- sum((data_subset$VO2_t - model(current_VO2_fast, current_tau_fast, new_VO2_slow, current_tau_slow, t_delay_slow))^2)
            if (new_rss < current_rss) {
              current_VO2_slow <- new_VO2_slow
              current_rss <- new_rss
              improved <- TRUE
            }
          }
          
          # Versuche tau_slow zu erhöhen
          new_tau_slow <- current_tau_slow + 0.1
          new_rss <- sum((data_subset$VO2_t - model(current_VO2_fast, current_tau_fast, current_VO2_slow, new_tau_slow, t_delay_slow))^2)
          if (new_rss < current_rss) {
            current_tau_slow <- new_tau_slow
            current_rss <- new_rss
            improved <- TRUE
          }
          
          if (!improved) break
        }
        
        if (current_rss < best_rss) {
          best_rss <- current_rss
          best_t_delay_slow <- t_delay_slow
          best_VO2_fast <- current_VO2_fast
          best_tau_fast <- current_tau_fast
          best_VO2_slow <- current_VO2_slow
          best_tau_slow <- current_tau_slow
        }
        
        incProgress(1 / num_iterations, detail = paste("Iteration", i, "von", num_iterations))
      }
    })
    
    if (!is.null(best_t_delay_slow)) {
      updateSliderInput(session, "time_delay_slow", value = best_t_delay_slow)
      updateSliderInput(session, "VO2", value = best_VO2_fast)
      updateSliderInput(session, "tau_fast", value = best_tau_fast)
      updateSliderInput(session, "VO2_slow", value = best_VO2_slow)
      updateSliderInput(session, "tau_slow", value = best_tau_slow)
      
      predicted <- VO2_Start + best_VO2_fast * (1 - exp(-(data_subset$t_s - t_delay) / best_tau_fast)) +
        best_VO2_slow * (1 - exp(-(pmax(data_subset$t_s - best_t_delay_slow, 0)) / best_tau_slow))
      r_squared <- calculate_r_squared(data_subset$VO2_t, predicted)
      current_r_squared(r_squared)
      
      showNotification(sprintf("Schritt 2 erfolgreich. R²: %.3f", r_squared), type = "message")
    } else {
      showModal(modalDialog(
        title = "Fit fehlgeschlagen",
        "Die Anpassung für die langsame Komponente konnte nicht erfolgreich durchgeführt werden.",
        easyClose = TRUE,
        footer = NULL
      ))
    }
  })
  
  observeEvent(input$fit_optimize, {
    req(current_data())
    Beispieldaten <- current_data()
    
    req(input$time_delay, input$VO2_Start, input$VO2, input$tau_fast, input$VO2_slow, input$tau_slow, input$time_delay_slow)
    
    t_delay <- input$time_delay
    VO2_Start <- input$VO2_Start
    
    data_subset <- Beispieldaten[Beispieldaten$t_s >= t_delay, ]
    
    # Werte aus Schritt 2
    VO2_step2 <- input$VO2
    tau_fast_step2 <- input$tau_fast
    VO2_slow_step2 <- input$VO2_slow
    tau_slow_step2 <- input$tau_slow
    t_delay_slow_step2 <- input$time_delay_slow
    
    # Berechne R² für Schritt 2
    step2_predicted <- VO2_Start + VO2_step2 * (1 - exp(-(data_subset$t_s - t_delay) / tau_fast_step2)) +
      VO2_slow_step2 * (1 - exp(-(pmax(data_subset$t_s - t_delay_slow_step2, 0)) / tau_slow_step2))
    step2_r_squared <- calculate_r_squared(data_subset$VO2_t, step2_predicted)
    
    best_fit <- NULL
    best_rss <- Inf
    
    withProgress(message = 'Optimierung läuft', value = 0, {
      for (i in 1:1000) {
        # Randomisierte Startwerte innerhalb der festgelegten Grenzen
        VO2_start <- runif(1, VO2_step2 * 0.98, VO2_step2 * 1.02)
        tau_fast_start <- runif(1, max(10, tau_fast_step2 - 3), tau_fast_step2 + 3)
        VO2_slow_start <- runif(1, VO2_slow_step2 * 0.98, VO2_slow_step2 * 1.02)
        tau_slow_start <- runif(1, max(90, tau_slow_step2 - 3), tau_slow_step2 + 3)
        t_delay_slow_start <- runif(1, max(0, t_delay_slow_step2 - 5), t_delay_slow_step2 + 5)
        
        start_values <- list(
          VO2 = VO2_start,
          tau_fast = tau_fast_start,
          VO2_slow = VO2_slow_start,
          tau_slow = tau_slow_start,
          t_delay_slow = t_delay_slow_start
        )
        
        fit <- try(nlsLM(VO2_t ~ VO2 * (1 - exp(-(t_s - t_delay) / tau_fast)) + 
                           VO2_slow * (1 - exp(-(t_s - t_delay_slow) / tau_slow)) + VO2_Start,
                         data = data_subset,
                         start = start_values,
                         lower = c(VO2 = VO2_step2 * 0.98, 
                                   tau_fast = max(10, tau_fast_step2 - 3), 
                                   VO2_slow = VO2_slow_step2 * 0.98, 
                                   tau_slow = max(90, tau_slow_step2 - 3), 
                                   t_delay_slow = max(0, t_delay_slow_step2 - 5)),
                         upper = c(VO2 = VO2_step2 * 1.02, 
                                   tau_fast = tau_fast_step2 + 3, 
                                   VO2_slow = VO2_slow_step2 * 1.02, 
                                   tau_slow = tau_slow_step2 + 3, 
                                   t_delay_slow = t_delay_slow_step2 + 5),
                         control = nls.control(maxiter = 1024)), silent = TRUE)
        
        if (!inherits(fit, "try-error")) {
          rss <- sum(residuals(fit)^2)
          if (rss < best_rss) {
            best_rss <- rss
            best_fit <- fit
          }
        }
        
        incProgress(1/1000, detail = paste("Iteration", i))
      }
    })
    
    if (!is.null(best_fit)) {
      params <- as.list(coef(best_fit))
      step3_predicted <- VO2_Start + params$VO2 * (1 - exp(-(data_subset$t_s - t_delay) / params$tau_fast)) +
        params$VO2_slow * (1 - exp(-(pmax(data_subset$t_s - params$t_delay_slow, 0)) / params$tau_slow))
      step3_r_squared <- calculate_r_squared(data_subset$VO2_t, step3_predicted)
      
      if (step3_r_squared > step2_r_squared) {
        updateSliderInput(session, "VO2", value = params$VO2)
        updateSliderInput(session, "tau_fast", value = params$tau_fast)
        updateSliderInput(session, "VO2_slow", value = params$VO2_slow)
        updateSliderInput(session, "tau_slow", value = params$tau_slow)
        updateSliderInput(session, "time_delay_slow", value = params$t_delay_slow)
        
        current_r_squared(step3_r_squared)
        showNotification(sprintf("Optimierung erfolgreich. Neues R²: %.3f", step3_r_squared), type = "message")
      } else {
        current_r_squared(step2_r_squared)
        showNotification(sprintf("Kein besserer Fit gefunden. Ergebnisse von Schritt 2 beibehalten. R²: %.3f", step2_r_squared), type = "warning")
      }
    } else {
      current_r_squared(step2_r_squared)
      showNotification(sprintf("Optimierung fehlgeschlagen. Ergebnisse von Schritt 2 beibehalten. R²: %.3f", step2_r_squared), type = "warning")
    }
  })
  
  output$plot <- renderPlotly({
    req(input$VO2, input$tau_fast, input$VO2_slow, input$tau_slow, input$VO2_Start, input$time_delay, input$time_delay_slow, input$VO2_Ruhe)
    
    VO2 <- input$VO2
    tau_fast <- input$tau_fast
    VO2_slow <- input$VO2_slow
    tau_slow <- input$tau_slow
    VO2_Start <- input$VO2_Start
    t_delay <- input$time_delay
    t_delay_slow <- input$time_delay_slow
    VO2_Ruhe <- input$VO2_Ruhe
    
    model_function <- function(t_s, VO2, tau_fast, VO2_slow, tau_slow, VO2_Start, t_delay, t_delay_slow) {
      VO2 * (1 - exp(-(pmax(t_s - t_delay, 0)) / tau_fast)) + 
        pmax(VO2_slow * (1 - exp(-(pmax(t_s - t_delay_slow, 0)) / tau_slow)), 0) + 
        VO2_Start
    }
    
    model_fast <- function(t_s, VO2, tau_fast, t_delay) {
      VO2 * (1 - exp(-(pmax(t_s - t_delay, 0)) / tau_fast))
    }
    
    model_slow <- function(t_s, VO2_slow, tau_slow, t_delay_slow) {
      pmax(VO2_slow * (1 - exp(-(pmax(t_s - t_delay_slow, 0)) / tau_slow)), 0)
    }
    
    Beispieldaten <- current_data()
    
    max_t <- if (!is.null(Beispieldaten) && nrow(Beispieldaten) > 0) max(Beispieldaten$t_s, na.rm = TRUE) else 300
    max_x <- max_t * 1.20
    t_s <- seq(0, max_x * 1.20, by = 1)
    
    model_values <- ifelse(t_s >= t_delay, model_function(t_s, VO2, tau_fast, VO2_slow, tau_slow, VO2_Start, t_delay, t_delay_slow), NA)
    fast_values <- ifelse(t_s >= t_delay, model_fast(t_s, VO2, tau_fast, t_delay), NA)
    slow_values <- ifelse(t_s >= t_delay_slow, model_slow(t_s, VO2_slow, tau_slow, t_delay_slow), NA)
    
    max_y <- if (!is.null(Beispieldaten) && nrow(Beispieldaten) > 0) max(Beispieldaten$VO2_t, na.rm = TRUE) * 1.05 else max(model_values, na.rm = TRUE) * 1.05
    
    eq_text <- sprintf("V̇O₂ = %.2f * (1 - e<sup>-(t - %.0f) / %.1f</sup>) + %.2f * (1 - e<sup>-(t - %.1f) / %.0f</sup>) + %.2f",
                       VO2, t_delay, tau_fast, VO2_slow, t_delay_slow, tau_slow, VO2_Start)
    t_halb <- tau_fast * log(2)
    
    shapes <- list(
      list(
        type = "line", x0 = t_delay, x1 = t_delay, y0 = 0, y1 = max_y * 1.2,
        line = list(color = "gray", width = 1, dash = "dash")
      ),
      list(
        type = "line", x0 = t_delay_slow, x1 = t_delay_slow, y0 = 0, y1 = max_y * 1.2,
        line = list(color = "gray", width = 1, dash = "dash")
      )
    )
    
    p <- plot_ly() %>%
      add_trace(x = ~t_s, y = ~model_values, type = 'scatter', mode = 'lines',
                name = 'V̇O2-Modellfunktion', line = list(color = '#EF6F6A')) %>%
      add_trace(x = ~t_s, y = ~fast_values, type = 'scatter', mode = 'lines', 
                name = 'V̇O<sub>2</sub><sub>fast</sub>', line = list(color = '#42BA97')) %>%
      add_trace(x = ~t_s, y = ~slow_values, type = 'scatter', mode = 'lines', 
                name = 'V̇O<sub>2</sub><sub>slow</sub>', line = list(color = '#BB7693')) %>%
      layout(title = "Biexponentielle V̇O<sub>2</sub>-Modellfunktion",
             margin = list(t = 40),
             xaxis = list(title = "t [s]", range = c(0, max_x)),
             yaxis = list(title = "V̇O<sub>2</sub> [l·min<sup>-1</sup>]", tickformat = ".1f"),
             shapes = shapes,
             annotations = list(
               list(
                 x = max_x * 0.35,
                 y = max_y * 0.3,
                 text = eq_text,
                 showarrow = FALSE,
                 xanchor = 'left',
                 yanchor = 'bottom',
                 font = list(
                   family = "Arial, sans-serif",
                   size = 10,
                   color = "black"
                 )
               ),
               list(
                 x = max_x * 0.35,
                 y = max_y * 0.2,
                 text = paste("T<sub>1/2</sub>:", round(t_halb, 1)),
                 showarrow = FALSE,
                 xanchor = 'left',
                 yanchor = 'bottom',
                 font = list(
                   family = "Arial, sans-serif",
                   size = 10,
                   color = "black"
                 )
               ),
               list(
                 x = t_delay, y = max_y * 0.9, text = sprintf("t<sub>delay</sub>: %.1f", t_delay), showarrow = FALSE, xanchor = "left", yanchor = "bottom",
                 textangle = -90, font = list(size = 11)
               ),
               list(
                 x = t_delay_slow, y = max_y * 0.9, text = sprintf("t<sub>delay_slow</sub>: %.1f", t_delay_slow), showarrow = FALSE, xanchor = "left", yanchor = "bottom",
                 textangle = -90, font = list(size = 11)
               )
             )) %>%
      add_trace(x = ~t_s, y = ~rep(VO2_Ruhe, length(t_s)), type = 'scatter', mode = 'lines',
                name = 'V̇O<sub>2, Ruhe</sub>', line = list(color = '#1CADE4'))
    
    if (show_data() && !is.null(Beispieldaten) && nrow(Beispieldaten) > 0) {
      data_subset <- Beispieldaten[Beispieldaten$t_s >= t_delay, ]
      
      model_values_at_data <- model_function(data_subset$t_s, VO2, tau_fast, VO2_slow, tau_slow, VO2_Start, t_delay, t_delay_slow)
      
      r_squared <- calculate_r_squared(data_subset$VO2_t, model_values_at_data)
      current_r_squared(r_squared)
      
      current_r_squared(r_squared)  # Aktualisieren Sie die reaktive Variable
      
      p <- p %>%
        add_trace(data = Beispieldaten, x = ~t_s, y = ~VO2_t, type = 'scatter', mode = 'markers+lines',
                  name = 'V̇O<sub>2</sub>', 
                  marker = list(color = 'rgba(38, 131, 198, 0.9)', size = 5.0),
                  line = list(color = 'rgba(38, 131, 198, 1.0)', width = 0.65, dash = '4 4'))
      
      p$x$layout$annotations <- c(p$x$layout$annotations,
                                  list(
                                    list(
                                      x = max_x * 0.35,
                                      y = max_y * 0.10,
                                      text = sprintf("R²: %.3f", current_r_squared()),
                                      showarrow = FALSE,
                                      xanchor = 'left',
                                      yanchor = 'bottom',
                                      font = list(
                                        family = "Arial, sans-serif",
                                        size = 10,
                                        color = "black"
                                      )
                                    )
                                  ))
    }
    
    p
  })
  output$instructions <- renderUI({
    HTML(
      "<div style='margin-top: 20px; padding: 10px; background-color: #f0f0f0; border: 1px solid #ddd; border-radius: 5px; width: fit-content;'>
      <h4 style='color: #333;'><strong>Anleitung - Modellanpassung:</strong></h4>
      <ol style='color: #555; list-style-position: outside; padding-left: 20px;'>
        <li>Beispiel-VO2-Daten einfügen oder eigene VO2-Daten als CSV-Datei hochladen.</li>
        <li>Zeitverzögerung_fast festlegen, um den Startpunkt der Modellanpassung zu bestimmen.</li>
        <li>Schrittweise die Modellanpassung durchführen:
          <ol>
            <li>Fit: Schnelle Komponente</li>
            <li>Fit: Langsame Komponente Verfeinerung</li>
            <li>Fit: Optimierung</li>
          </ol>
        </li>
        <li>Alternative: Manuelle Anpassung der Modellparameter über die Schieberegler.</li>
      </ol>
      <div style='margin-top: 20px;'></div>
      <pre style='background-color: #f8f8f8; padding: 10px; border: 1px solid #ddd; border-radius: 5px; width: fit-content;'>
VO2-Daten können als CSV-Datei im folgenden Format hochgeladen werden:
t_s,VO2_t
0.0,0.479
1.0,0.459
2.3,0.488
…
      </pre>
    </div>"
    )
  })
}

# App ausführen
shinyApp(ui = ui, server = server)


```  



::: {.callout-note icon=false collapse=true appearance="simple"}
### Modellanpassung - Details:

**1. Schnelle Komponente (1. Fit):**

- Verwendete Funktion: nlsLM
- Parameter: V̇O2_fast, tau_fast
- Grenzen:
  - V̇O2_fast: *0.5 bis 7.0 l/min*
  - tau_fast: *10 bis 120 s*
- Prozess: *100 Iterationen* mit zufälligen Startwerten
- Optimierung: Minimierung der Residuenquadratsumme (RSS)

**2. Langsame Komponente & Verfeinerung (2. Fit):**

- Parameter: V̇O2_fast, tau_fast, V̇O2_slow, tau_slow, t_delay_slow
- Initiale Werte (basierend auf Ozyener et al., 2001):
  - V̇O2_slow: *0.4 l/min*
  - tau_slow: *160 s*
- Grenzen:
  - V̇O2_slow: *0.05 bis 1.0 l/min*
  - tau_slow: *90 bis Unendlich s*
  - t_delay_slow: *tau_fast + t_delay bis max(t_s)*
- Prozess für t_delay_slow:
  a) Berechnung des Suchbereichs: von *(tau_fast + t_delay)* bis zum maximalen Zeitpunkt der Daten
  b) Iteration durch jeden möglichen Wert in diesem Bereich (*1-Sekunden-Schritte*)
  c) Für jeden t_delay_slow-Wert:
     - Berechnung der RSS mit festen Werten für V̇O2_slow (*0.4 l/min*) und tau_slow (*160 s*)
  d) Auswahl des t_delay_slow-Wertes mit der niedrigsten RSS
- Optimierung von V̇O2_fast und tau_fast:
  a) Schrittweise Reduzierung von V̇O2_fast (*0.01 l/min*) und tau_fast (*0.1 s*)
  b) Bei jeder Änderung: Berechnung der neuen RSS
  c) Akzeptanz der Änderung nur bei Verbesserung der RSS
  d) Wiederholung bis keine Verbesserung mehr möglich
- Optimierung von V̇O2_slow und tau_slow:
  a) Zunächst Reduzierung: V̇O2_slow (*0.01 l/min*) und tau_slow (*0.1 s*)
  b) Dann Erhöhung: V̇O2_slow (*0.01 l/min*) und tau_slow (*0.1 s*)
  c) Bei jeder Änderung: Berechnung der neuen RSS
  d) Akzeptanz der Änderung nur bei Verbesserung der RSS
  e) Wiederholung jeder Richtung bis keine Verbesserung mehr möglich

**3. Optimierung (3. Fit):**

- Verwendete Funktion: nlsLM
- Parameter: V̇O2_fast, tau_fast, V̇O2_slow, tau_slow, t_delay_slow
- Grenzen: Basierend auf den Ergebnissen von Schritt 2
  - V̇O2_fast: *±2% des Wertes aus Schritt 2*
  - tau_fast: *±3 s des Wertes aus Schritt 2 (Minimum 10 s)*
  - V̇O2_slow: *±2% des Wertes aus Schritt 2*
  - tau_slow: *±3 s des Wertes aus Schritt 2 (Minimum 90 s)*
  - t_delay_slow: *±5 s des Wertes aus Schritt 2 (Minimum 0 s)*
- Prozess: *1000 Iterationen* mit zufälligen Startwerten innerhalb der Grenzen
- Optimierung: Minimierung der RSS

:::



## Umgang mit Daten in dieser Arbeit --> In Methodenteil
In dieser ARbeit wurde der jewielige Anstieg des $\dot{V}O_{2}$ nach Belastungsbeginn für jede Belastungsstufe wiefolgt bearbeitet.  zunächst wurden die aus den  Atemzug-für-Atemzug $\dot{V}O_{2}$-Daten die Ausreißer entfernt wurden, dann durch einen gleitenden 3-Atemzug-Durchschnitt geglättet und anschließend  der Kurvenanpassung unterzogen wurden. Vor der Kurvenanpassung wurden die Daten bearbeitet, indem die zeitverzögerung sogeählt wurde das die Phase-I-Daten der kardiorespiratorischen Antwort nicht in dei Kurvenanpassung einezogen wurden. Die Kurvenanpassung wurde dann mit @eq-VO2_Start2 an die jeweiligen datne durchgeführt. Die Modellparameter der Kurvenanpassung wurden wie olgt gewählt:


::: {.callout-note icon=false collapse=true appearance="simple"}
### Parameter - Kurvenanpassung:
- Verwendete Funktion: nlsLM
  * Maximale Iterationen: *1024*
  * Verwendung des Levenberg-Marquardt-Algorithmus
- Parameter: V̇O2 (Amplitude), Tau
- Modell: V̇O2_t ~ V̇O2 · (1 - exp(-(t_s - t_delay) / Tau)) + V̇O2_Start
- Grenzen:
  * V̇O2: *0.5 bis 7.0 l/min*
  * Tau: *5 bis 360 s*
- Optimierung: Minimierung der Residuenquadratsumme (RSS)
:::





### Kritik und Gründe --> Glaube aus Stiling oder Siestema
Diskussionen über die Ursache der langsamen Komponente haben ergeben, dass das Blutlaktatprofil zwar gut mit ihrer zeitlichen Entwicklung korreliert, die Beziehung jedoch eher zufällig als kausal ist. Der wahrscheinlichste Mechanismus ist die Rekrutierung von Muskelfasern [Gaesser und Poole 1996]. Billat (2000) definiert die kritische Geschwindigkeit, v~crit~, als die höchste Geschwindigkeit, bei der V̇O~2~ einen verzögerten, submaximalen Steady State erreichen kann. Hill und Ferguson (1999) zeigten, dass "v~crit~ die Schwellenintensität ist, oberhalb derer Belastung von ausreichender Dauer zum Erreichen von V̇O~2max~ führt". Dies wird verwendet, um die untere Grenze für schwere Belastung als "knapp über v~crit~, was die höchste Arbeitsrate ist, die über einen längeren Zeitraum aufrechterhalten werden kann und nicht V̇O~2max~ hervorruft" zu definieren.

Die V̇O~2~ langsame Komponente sollte nicht mit dem Phänomen der V̇O~2~-Drift verwechselt werden, einem langsamen Anstieg des V̇O~2~ während längerer, submaximaler Belastung mit konstanter Leistungsabgabe. Im Gegensatz zur langsamen Komponente wird die V̇O~2~-Drift bei Leistungen weit unterhalb der Laktatschwelle beobachtet, und das Ausmaß des V̇O~2~-Anstiegs ist viel geringer [Stirling2015].








- Untersuchungen haben gezeigt, dass der "Gain" (VI~2~/∆WR) oberhalb der Laktatschwelle höher ist als bei moderater Belastung [@BarstowMole1991; @Zoladz1995; @TschakovskyHughson1999], was auf einen höheren O~2~-"Verbrauch" pro Arbeitsrateninkrement hinweist.


- Cerretelli (1987) zeigte, dass bei quadratischer Belastung eine anfängliche schnelle Komponente der V̇O~2~- und V̇CO~2~-Anschaltreaktionen existiert, die etwa 30% der gesamten Steady-State-Änderung ausmacht [@Cerretelli1987.315].



-Aus @Barstow1994: Die vorherrschende Phase-2-Reaktion steigt weiterhin exponentiell an, mit etwa der gleichen Zeitkonstante wie bei moderater Belastung, und ihre Amplitude bleibt linear zur Belastungsintensität. Allerdings wird in der Regel auch ein zusätzlicher, sich langsam entwickelnder Anstieg des VO2 beobachtet, der etwa 100-200 Sekunden nach Beginn der Belastung einsetzt. Dieser zusätzliche VO2-Anstieg verzögert das Erreichen eines Steady-States, erhöht den gesamten Sauerstoff-"Bedarf" der Übung und steht in statistischem Zusammenhang mit der Geschwindigkeit und dem Ausmaß des Blutlaktatanstiegs.

@Oezyner2001 fanden, dass der Wert von τ~Start~ für die Sauerstoffaufnahme im Durchschnitt bei 33 ± 16 s liegt, unabhängig von der Belastungshöhe. 
-Im Gegensatz dazu berichten andere Studien von einem Anstieg von τ~Start~ mit zunehmender Belastung (Linnarsson 1974).
Wasserman et al. [@Wasserman1967] prognostizierten und fanden experimentell heraus, dass der Sauerstoffverbrauch bei schweren Arbeitsbelastungen länger ansteigt als bei moderaten Intensitäten. Henry und DeMoor [@Henry1956] stellten ebenfalls fest, dass die Zeit bis zum Erreichen eines stabilen Zustands der Sauerstoffaufnahme (VO2) mit steigender Arbeitslast zunimmt. Im Gegensatz dazu berichten Di Prampero et al. [@DiPrampero1970] und Margaria et al. [@Margaria1965], dass das Muster des VO2-Anstiegs zum stabilen Zustand unabhängig von der Arbeitsintensität gleich ist und dieser Anstieg durch einen einzigen exponentiellen Prozess beschrieben wird, wobei die Reaktionskonstante für alle Arbeitslasten gleich ist.



![Verlauf VO2 - leicht bis schwer (klein)](Ozyener_ON_small.png){#fig-Verlauf_ON2}


## Verhalten bei verschiedenen Intensitäten


::: columns
::: {.column width="59%"}

**Moderate Intensität:**
Die pulmonale V̇O~2~steigt monoexponentiell und erreicht innerhalb von etwa 3 Minuten einen Steady State [@Whipp1982; @Hughson1988; @Whipp1994]. Die V̇O~2~ erhöht sich linear zur Arbeitsrate um etwa 10 ml·min^-1·W^-1. Sobald der Steady State erreicht ist, sind ATP-Resynthese und oxidative Phosphorylierung im Gleichgewicht [@Kindermann1979; @Faude2009].

**Schwere Intensität:**
Die schwere Intensität liegt zwischen der maximalen Leistungsfähigkeit (GXT) und der kritischen Leistung (CP) [@Hill1993]. Bei schwerer Belastung ist der oxidative Bedarf höher, und der Steady State verzögert sich durch die V̇O~2~ Slow Component. Die O~2~-Kosten steigen auf etwa 13 ml·min^-1·W^-1 [@WhippWasserman1972; @PearceMilhorn1977; @WhippMahler1980; @BarstowMole1991]. Die arterielle Laktatkonzentration steigt über die Ausgangswerte, erreicht aber ein erhöhtes Gleichgewicht [@Billat2003].

**Schwere Intensität:**
In der Forschung ist umstritten, ob die primäre V̇O~2~-Kinetik bei schwerer Belastung langsamer ist als bei moderater. Dies könnte auf eine größere Zeitkonstante oder eine zusätzliche Slow Component in V̇O~2~ hindeuten, die einen verzögerten Steady State über dem vorhergesagten Niveau erreicht [@PatersonWhipp1991; @BarstowMole1991].

**Sehr schwere Intensität:**
Bei sehr schwerer Intensität, oberhalb des maximalen Laktat-Steady-State oder der kritischen Leistung, erreichen weder V̇O~2~ noch arterielle Laktatkonzentration einen Steady State [@GaesserPoole1996]. Beide steigen unaufhaltsam, bis Ermüdung eintritt und maximale V̇O~2~-Werte erreicht werden [@WassermanWhipp1975; @GaesserPoole1996].
:::

::: {.column width="2%"}
:::

::: {.column width="39%"}
![Beispiele der VO2-Anstiegskinetik bei einem Probanden für unterschiedliche Intensitäten: sehr intensiv, schwer, mittelschwer und moderat (von oben nach unten). Unterhalb der VO2-Daten ist der zugehörige Residuenplot zu sehen. In Spalte A ist das mono-exponentielle Modell und in Spalte B das doppel-exponentielle Modell dargestellt (Özyener 2001).](Ozyener_ON.png){#fig-Verlauf_ON}
::: 
:::















########################




# Ozyener 2001
 Oberhalb von θ~L~ ist die V̇O~2~-Kinetik komplexer [@WhippWasserman1972; @Linnarsson1974; @HughsonMorrissey1982; @PatersonWhipp1991; @BarstowMole1991]. Mehrere Gruppen haben gezeigt, dass der "Gain" (d.h. ∆V̇O~2~/∆WR (Arbeitsrate)) oberhalb von θ~L~ den für moderate Belastung übersteigt [@WhippMahler1980; @Casaburi1987; @Roston1987; @BarstowMole1991; @PatersonWhipp1991; @Zoladz1995; @TschakovskyHughson1999], was einen höheren O~2~-"Verbrauch" pro Arbeitsrateninkrement widerspiegelt. 
Diese supra-θ~L~ V̇O~2~-Antwort besteht aus zwei Komponenten: (a) einer "fundamentalen" exponentiellen Phase und (b) einer nachfolgenden Phase mit verzögertem V̇O~2~-Beginn, die zu einem sich langsam entwickelnden zusätzlichen Anstieg der V̇O~2~ führt, was als "Excess" V̇O~2~ bezeichnet wurde [@Whipp1987]. Allerdings wurden weder die kinetischen Merkmale noch die Determinanten dieser zusätzlichen V̇O~2~-Komponente in Bezug auf spezifische Belastungsintensitätsbereiche hinreichend etabliert.

Die V̇O~2~-Kinetik beim Belastungsabbruch für moderate Belastung wurde ebenfalls durch ein Modell erster Ordnung charakterisiert, ähnlich dem der Belastungsaufnahme, mit einer einzigen Zeitkonstante (τ), Verzögerung (d) und Amplitude (A) [@Linnarsson1974; @PatersonWhipp1991]. Die supra-θ~L~ Kinetik beim Belastungsabbruch wurde jedoch als schneller als die der Belastungsaufnahme beschrieben [@Cerretelli1977; @PatersonWhipp1991; @Gerbino1996; @MacDonald1997; @LangsetmoPoole1999]. Interessanterweise wurde festgestellt, dass diese Antwort beim Belastungsabbruch entweder durch eine monoexponentielle Funktion oder eine doppelt exponentielle Funktion mit einer langsamen Komponente gut beschrieben werden konnte, die deutlich kleiner in der Amplitude war als bei der Belastungsaufnahme. Das heißt, die V̇O~2~-Kinetik beim Belastungsabbruch kann Charakteristiken erster Ordnung beibehalten, obwohl die Arbeitsrate θ~L~ überschreitet [@Linnarsson1974; @PatersonWhipp1991; @Gerbino1996; @Bohnert1998; @LangsetmoPoole1999].

Da Symmetrie ein wesentliches Merkmal linearer Kontrollsystemdynamik ist [@Milsum1966], waren wir daran interessiert, den Grad zu untersuchen, in dem die "An-Aus"-Symmetrie der V̇O~2~-Kinetik für eine Reihe physiologisch definierter Belastungsintensitäten erhalten blieb (oder nicht). Die a priori Zuweisung der Belastungsintensitäten war: (a) moderat (sub-θ~L~); (b) schwer, für die ein Steady-State sowohl in der erhöhten arteriellen Blutlaktatkonzentration ([L^-^]) als auch in der V̇O~2~-Antwort erwartet wird; (c) sehr schwer, für die kontinuierliche Anstiege sowohl in der [L^-^] als auch in der V̇O~2~-Antwort erwartet werden, und (d) extrem, was Arbeitsraten widerspiegelt, die Steady-State V̇O~2~-Werte über dem V̇O~2,peak~ erfordern.

# Cerretelli1987 - S.315
Trotz einiger spezifischer Unterschiede zeigen die meisten der oben genannten Studien, dass als erste Annäherung sowohl die V̇CO~2~- als auch die V̇O~2~-Kinetik durch monoexponentielle Funktionen beschrieben werden können, mit t~1/2~ = 35-50 s für V̇CO~2~ und 20-30 s für V̇O~2~, wenn die Kinetik der An- und Abschaltreaktionen während quadratischer Belastungsänderungen im Wesentlichen gleich ist. Wie Linnarsson hervorhob, kann die Analyse der V̇O~2~- und V̇CO~2~-Kinetik wesentlich verfeinert werden, indem Zeitverzögerungen und/oder Exponentialfunktionen zweiter Ordnung einbezogen werden. Mit diesem Ansatz war es bei quadratischer Belastung möglich, die Existenz einer anfänglichen schnellen Komponente der V̇O~2~- und V̇CO~2~-Anschaltreaktionen nachzuweisen, die etwa 30% der gesamten Steady-State-Änderung ausmacht.





## Historical background -> Clark2014
Sauerstoffaufnahme-Kinetik untersucht die Veränderungsrate der pulmonalen Sauerstoffaufnahme (V̇O~2~) zu Beginn oder Ende körperlicher Belastung. Die Reaktion unterscheidet sich oberhalb und unterhalb der Gasaustauschschwelle (GXT) und verläuft phasenweise, abhängig von der relativen Belastungsintensität [@Hughson1982; @Hughson1990; @Barstow1994; @JonesPoole2005].


## Ferretti2023 S.109 ff noch anschauen


# Sauerstoffaufnahmekinetik bei leichter bis moderater körperlicher Belastung -> Waserman2020

Der Anstieg von V˙O2 während der Belastung spiegelt sowohl die Reaktionen des Herzminutenvolumens als auch des C(a − )O2 wider. Wenn während der Phase I einer moderaten Belastung V˙O2 doppelt so hoch wäre wie der Ruhewert und C(a − )O2 nicht ansteigen würde, könnte man daraus schließen, dass das Herzminutenvolumen sich verdoppelt hat. Aus diesem Grund wurde die Phase-I-V˙O2-Antwort auch als kardiodynamische Phase bezeichnet. Während einer aufrechten Belastung, die aus sitzender Ruhe beginnt, zeigte Casaburi et al. jedoch, dass etwa ein Drittel des Phase-I-V˙O2 durch einen abrupten Anstieg des C(a − )O2 zu Beginn der Belastung erklärt werden kann. Dies ist offenbar auf eine Blutstase während der Ruhe in aufrechter Haltung zurückzuführen; wenn die Stase durch elastische Bandagen an den Beinen verhindert wird, ist dieser abrupte Anstieg des C(a − )O2 nicht erkennbar.

Veränderungen der Lungengasspeicher haben aus zwei Gründen relativ wenig Einfluss auf die V˙O2-Kinetik: Die alveolären O2- (und CO2-) Konzentrationen ändern sich sehr wenig, und bei normalen Personen ist jede Abnahme des endexspiratorischen Lungenvolumens (EELV) in der Regel moderat (d.h. typischerweise weniger als 0,5 L) und tritt ziemlich schnell auf. Da 0,5 L alveoläres Gas etwa 75 mL O2 enthalten, hätte eine Änderung des EELV während Phase I oder II nur einen geringen Einfluss auf die Gasaustauschkinetik, selbst wenn sie schnell auftreten würde. Allerdings tragen Atemzug-zu-Atemzug-Änderungen im EELV zu den beobachteten Atemzug-zu-Atemzug-Schwankungen in V˙O2 (und V˙CO2) bei, was es schwieriger macht, ihre Kinetik genau zu quantifizieren. Trotz der relativ geringen Änderungen der Lungengasspeicher zu Beginn der Belastung wurde ein Ansatz beschrieben, um sowohl die Änderung der alveolären Konzentration als auch die Änderung des Lungenvolumens für die Untersuchung der V˙O2- und V˙CO2-Kinetik zu korrigieren.

Studien zur V˙O2-Kinetik konzentrieren sich häufig auf die vorherrschende Phase-II-Komponente, weshalb solche Studien oft von einer Belastungsbaseline aus durchgeführt werden; das heißt, um den Einfluss von Phase I zu reduzieren. Während Phase II entspricht V˙O2 dem von den Zellen verbrauchten O2 plus den Kreditoren der O2-Schuld. Dazu gehören die Abnahme des O2-Gehalts im venösen Blut, physikalisch gelöstes O2, Änderungen der [Oxymyoglobin] in den kontrahierenden Muskeln, die Abnahme des Muskel-[PCr], anaerobe Glykolyse, die zur Akkumulation von Laktat führt (auch vorübergehend), und jede Änderung der Lungen-O2-Speicher.

Die resultierende Phase-II-V˙O2-Antwort wird gut durch eine einzelne Exponentialfunktion charakterisiert, mit einer Zeitkonstante (d.h. der Zeit, um 63% der endgültigen Antwort zu erreichen) von etwa 30 Sekunden bei gesunden Personen. Ein Steady-State in V˙O2 wird erreicht und kann nur dann aufrechterhalten werden, wenn alle zellulären Energieanforderungen aus Reaktionen stammen, die O2 aus der Atmosphäre verwenden. Die Zeitkonstante der V˙O2-Antwort kann bei dekonditionierten Personen, gesunden älteren Menschen und bei chronischen Herz-, Lungen- oder Nierenerkrankungen signifikant erhöht sein (d.h. die Kinetik verlangsamt). Die Mechanismen, die zu dieser verlangsamten kinetischen Antwort führen, können bei diesen Zuständen unterschiedlich sein, aber sie führen alle zu einer verzögerten Erreichung des Steady-States und damit zu einem größeren O2-Defizit für eine gegebene Belastungsstufe. Verlangsamte V˙O2-Kinetik bei Alterung oder chronischen Erkrankungen versetzen den beteiligten Skelettmuskel für eine gegebene Belastungsstufe in einen niedrigeren energetischen Zustand, da ein großer Teil des O2-Defizits aus PCr-Abbau besteht. Ausdauertraining zur Erhöhung der oxidativen Kapazität des Skelettmuskels kehrt diesen Effekt um, beschleunigt die V˙O2-Kinetik und reduziert das O2-Defizit der Belastung.

Normalerweise wird die Phase-II-V˙O2-Kinetik weitgehend durch intramuskuläre Prozesse bestimmt, die zu Beginn der Belastung mit der Spaltung energiereicher Phosphatbindungen des vorhandenen ATP eingeleitet werden, um den unmittelbaren Energiebedarf des kontrahierenden Muskels zu decken. Das resultierende ADP wird sofort durch PCr-Abbau zu ATP rephosphoryliert. [PCr] nimmt daher schnell auf ein neues Niveau ab, das proportional zur Belastungsstufe ist, und kehrt innerhalb der ersten Minuten der Erholung auf das Ruheniveau vor der Belastung zurück. Die anaeroben Mechanismen der ATP-Regeneration während Belastungen moderater Intensität - Spaltung von intramuskulärem PCr und in geringerem Maße anaerobe Glykolyse - funktionieren nur vorübergehend, bis die mitochondriale oxidative Phosphorylierung vollständig rekrutiert wird und ein V˙O2-Steady-State erreicht wird. Während die PCr-Hydrolyse jedoch sofort zu Beginn der Belastung einsetzt, sind die PCr-Vorräte in ihrer Menge begrenzt und werden erst bei der Rückkehr in den Ruhezustand wieder aufgefüllt. Das PCr wird durch aerobe (O2-benötigende) Mechanismen in der Erholung wiederhergestellt und macht einen Großteil der O2-Schuld aus. Experimentelle Beweise deuten darauf hin, dass die Zunahme der intramuskulären Konzentration von Pi und insbesondere von ADP die oxidative Phosphorylierung stimuliert, wodurch ATP wieder aufgefüllt und die Muskel-[ATP] auf dem Ruheniveau gehalten wird, außer unter extremsten Bedingungen.

Die Phase-II-V˙O2-Antwort ist eng mit der Kinetik des Muskel-PCr-Abbaus verbunden. Dies führt zur intramuskulären Akkumulation von ADP und Pi, die wichtige Signale für die mitochondriale oxidative Phosphorylierung und damit für Q˙O2 und V˙O2 während der Belastung sind; ATP wird daher wieder aufgefüllt, wodurch die Muskel-[ATP] relativ konstant bleibt. Diese V˙O2-Antwort wird deutlich beschleunigt, wenn der PCr-Abbau durch pharmakologische Blockade der Kreatinkinase gehemmt wird, wobei ADP und Pi extrem schnell akkumulieren. Daher ist die mitochondriale oxidative Kapazität (der funktionelle Stellvertreter des Muskelmitochondriengehalts) ein Schlüsselfaktor für die V˙O2-Kinetik. So führt Belastung, die sich ausschließlich auf Typ-I-Fasern stützt, die einen höheren Mitochondriengehalt aufweisen, zu einer schnelleren V˙O2-Kinetik als Belastung, die sowohl Typ-I- als auch Typ-II-Fasern einbezieht. Obwohl die Bereitstellung der reduzierenden Äquivalente NADH + H+ und O2 an das mitochondriale Elektronentransportsystem jeweils das Potenzial hat, die Anstiegsrate von V˙O2 zu Beginn der Belastung zu begrenzen, scheint ihre Bereitstellung normalerweise im Überfluss vorhanden zu sein und nicht die V˙O2-Kinetik zu begrenzen. Dennoch könnten diese Substrate unter bestimmten Bedingungen limitierend werden, zum Beispiel während des Alterns, wo die Aktivierung von PDH verlangsamt ist, oder bei Herzinsuffizienz, wo der Muskelkapillar-PO2 zu Beginn der Belastung schnell abfällt.

# Sauerstoffaufnahmekinetik bei hoher körperlicher Belastung --> Wassermann2020 - Gas Exchange Kinetics


Während Belastungen oberhalb der anaeroben Schwelle (AT) wird die V˙O2-Kinetik komplexer. Bei Belastungen hoher Intensität (d.h. etwas über der AT) kann V˙O2 einen Steady-State erreichen, allerdings mit einem sehr verzögerten Zeitverlauf. Es kann bis zu 15 Minuten dauern, bis sich V˙O2 stabilisiert, und mit einer erhöhten Steady-State-Amplitude (d.h. mit einem Anstieg von etwa 11-13 mL/min/W). Wie oben beschrieben, ist Belastung in diesem Bereich durch eine erhöhte, aber stabile arterielle [Laktat]-Konzentration gekennzeichnet. Bei höheren Belastungsstufen (im Bereich sehr hoher Intensität) werden jedoch selten Steady-States in V˙O2 und [Laktat] erreicht, wobei der V˙O2-Anstieg am Ende der Belastung bis zu 13 bis 14 mL/min/W erreichen kann.

Die Schwelle, die bestimmt, ob eine Belastungsstufe in einem verzögerten Steady-State (schwer) aufrechterhalten werden kann oder ob ein Steady-State nie erreicht werden kann (sehr schwer), wird als kritische Leistung (CP) bezeichnet. Die CP ist eine wichtige funktionelle Schwelle mit erheblichen Auswirkungen auf die Belastungstoleranz. Bei gesunden Personen führt Belastung oberhalb der CP dazu, dass V˙O2 unaufhaltsam ansteigt, solange die Belastungsstufe aufrechterhalten wird. Unter diesen Bedingungen wird V˙O2 schließlich V˙O2max erreichen, zu diesem Zeitpunkt steigt der V˙O2-Bedarf weiter an, während V˙O2 selbst am Maximum begrenzt ist: Dies schafft die Bedingungen für Intoleranz. Noch höhere Belastungsstufen, bei denen der O2-Bedarf der Belastung von Anfang an V˙O2max übersteigt (schwere Belastung), sind jedoch mit einem reduzierten V˙O2-Anstieg verbunden. Dies liegt daran, dass V˙O2 den Maximalwert nicht überschreiten kann und daher nie seinen Bedarf erreichen kann. Solche Belastungsstufen sind typischerweise nur für etwa 2-3 Minuten aufrechtzuerhalten.

Die zusätzliche V˙O2-Antwort, die zu einem erhöhten oder "überschüssigen" O2-Verbrauch bei der Arbeit führt, wurde als V˙O2-Slow-Komponente bezeichnet, da sie langsam einsetzt (sie wird erst etwa 2-3 Minuten nach Belastungsbeginn beobachtet) und sich langsam im Vergleich zur Phase-II-V˙O2-Kinetik entwickelt. Die Anstiegsrate von V˙O2 während der Slow-Komponenten-Phase korreliert mit der Größe des entsprechenden arteriellen [Laktat]-Anstiegs.

Es gibt einige Diskussionen über die Mechanismen, die zur V˙O2-Slow-Komponente beitragen. Sie spiegelt größtenteils (d.h. etwa 85%) einen erhöhten O2-Verbrauch bei der Arbeit in den aktiven Bewegungsmuskeln wider, wobei der Rest vermutlich Erhöhungen der Herz- und Atemarbeit widerspiegelt. Obwohl die V˙O2-Slow-Komponente mit Laktatakkumulation verbunden ist, verursacht Laktat an sich nicht die Slow-Komponente. Vielmehr ist der am konsistentesten unterstützte Kandidat für die Ursache der V˙O2-Slow-Komponente eine reduzierte Muskeleffizienz infolge der fortschreitenden Rekrutierung von mehr niedrigeffizienten schnell zuckenden Muskelfasern im Verlauf der Belastung. Dennoch war es schwierig, eine fortschreitende Typ-II-Faser-Rekrutierung nachzuweisen, obwohl Studien zur Muskelbildgebung mittels Magnetresonanz und zur Glykogenentleerung in Typ-II-Muskelfasern diesen Vorschlag unterstützen. Darüber hinaus scheint Muskelermüdung eine obligatorische Voraussetzung für die V˙O2-Slow-Komponente zu sein, wobei das Ausmaß der Muskelermüdung mit dem Ausmaß der Slow-Komponente korreliert. Somit spiegelt die V˙O2-Slow-Komponente wahrscheinlich eine fortschreitende Ineffizienz der Muskelkontraktionen wider, die aus Muskelermüdung und zusätzlicher Muskelfaserrekrutierung resultiert, um die vorgegebene Belastungsstufe aufrechtzuerhalten, was zu einem erhöhten ATP-Bedarf führt, ohne die Fähigkeit, den V˙O2-Anforderungen der Aufgabe in einem Steady-State gerecht zu werden.

Besonders relevant für kardiopulmonale Belastungstests (CPET) ist die Betrachtung, wie V˙O2 auf das relativ schnelle inkrementelle Belastungsprofil von 8 bis 12 Minuten Dauer reagiert. Nach einer kurzen verzögerungsähnlichen Phase steigt V˙O2 gleichmäßig und linear an, wenn die Belastungsstufe relativ schnell erhöht wird, entweder in einem kontinuierlichen Rampenmuster oder inkrementell in gleichen Schritten von 1 Minute Dauer. Die Steigung der linearen Phase der resultierenden Beziehung zwischen V˙O2 und Belastungsstufe (ΔV˙O2/ΔWR) beträgt normalerweise etwa 10 mL/min/W und ist ähnlich dem sub-AT-Steady-State-V˙O2-Anstieg. Im Kontext der Berechnung der Arbeitseffizienz sollte die numerische Ähnlichkeit der Steady-State- und Rampen-/inkrementellen V˙O2-Anstiege jedoch mit Vorsicht interpretiert werden. Das heißt, die Berechnung des kalorischen Äquivalents von V˙O2 für eine bestimmte Belastungsstufe erfordert, dass sowohl V˙O2 als auch V˙CO2 Steady-States erreicht haben, was bei Rampen- oder inkrementellen Protokollen nicht der Fall ist.

Dieses verzögert-lineare Verhalten von V˙O2 bei Rampen- oder inkrementeller Belastung ist vorhersehbar aus der monoexponentiellen Kinetik der vorherrschenden Phase-II-V˙O2-Antwort auf moderate konstante Belastung, so dass V˙O2 der Steady-State-V˙O2-Antwort um ein konstantes Intervall nachhinkt (das der mittleren Antwortzeit, MRT, entspricht). Dass die V˙O2-Antwort ihre lineare Trajektorie oberhalb der AT beibehält, anstatt steiler zu werden, ist eine Reflexion der langsamen Rekrutierungskinetik der V˙O2-Slow-Komponente, relativ zur vorgegebenen Belastungssteigerungsrate. Bei sehr langsamen Belastungssteigerungsraten wurde gezeigt, dass die V˙O2-WR-Beziehung oberhalb der AT steiler wird, da ausreichend Zeit für die Entwicklung der V˙O2-Slow-Komponente vorhanden ist.








## Modellfunktion des Sauerstoffvolumenstroms in Abhängigkeit von der Zeit während der Belastung





## Bi-exponentiale Modellfunktion des Sauerstoffvolumenstroms in Abhängigkeit von der Zeit während der Belastung




# Anpassbarer und ausführbarer R-Code mit webR eines des Sauerstoffvolumenstrom-Verlaufs in Abhängigkeit von der Zeit während der Belastung

```{webr-r}
# Hier VO2-Daten einfügen

# Zeitpunkte
t_s <- c(0, 2.8, 5.3, 8.6, 10.4, 15.2, 19.7, 22.5, 24.5, 26.2, 28.2, 31.6, 33.5, 35.3, 37, 38.5, 40.3, 41.9, 43.5, 45.2, 46.8, 48.4, 50, 51.5, 53.3, 54.8, 56.5, 57.9, 59.5, 61.1, 62.9, 64.3, 65.8, 67.7, 69.1, 70.7, 72.4, 73.9, 75.7, 77.2, 78.9, 80.5, 82.7, 84.1, 85.5, 87, 88.6, 90.2, 91.5, 92.9, 94.3, 95.8, 97.6, 99.1, 100.5, 102.1, 103.5, 105.3, 106.6, 108.1, 109.6, 111.4, 112.8, 114.3, 115.8, 117.7, 119.1, 120.8, 122.3, 124.2, 125.7, 127.2, 128.8, 131.1, 132.5, 133.9, 135.4, 136.9, 138.4, 140, 141.3, 143, 144.5, 146.1, 147.6, 149, 150.5, 152.3, 153.9, 155.6, 156.9, 158.6, 160.1, 162.4, 163.7, 165.1)

# VO2-Werte
VO2_t <- c(0.465, 0.34, 0.466, 0.503, 0.452, 0.687, 0.846, 0.963, 0.936, 1.125, 1.238, 1.541, 1.395, 1.685, 1.689, 1.622, 1.917, 1.732, 1.835, 1.801, 1.664, 1.744, 1.891, 1.85, 1.858, 1.89, 2.108, 1.962, 1.992, 1.972, 2.026, 1.868, 1.949, 2.033, 1.887, 1.856, 2.112, 2.117, 2.129, 1.994, 2.03, 1.925, 1.916, 1.949, 1.956, 2.119, 1.894, 2.034, 1.766, 1.994, 1.985, 2.068, 1.796, 2.172, 2.09, 2.017, 2.152, 2.238, 2.027, 2.214, 2.159, 2.098, 1.963, 2.063, 2.292, 2.21, 1.84, 2.102, 2.1, 2.086, 2.008, 2.048, 1.978, 1.87, 2.16, 1.977, 2.335, 2.053, 2.084, 2.056, 2.005, 2.161, 2.088, 2.161, 2.078, 2.038, 2.117, 2.148, 2.043, 2.342, 2.142, 2.306, 2.043, 1.881, 1.918, 2.252)

plot(t_s, VO2_t, type="o", col="blue", xlab="Zeit [s]", ylab="VO2 [l/min]", main="VO2 [l/min]")

```
